# Generowanie danych dla benchmarków

Obliczenia pokazane tutaj należy wykonać używając podstawowego środowiska, które zostało przygotowane. Potrzebne informacje znajdują się w pliku [Setup](../Setup.ipynb).

In [3]:
# Obliczenia potrzebne do benchmarku naszej implementacji symulowanej bifurkacji

import os
import time
import pickle

import cupy as cp
import matplotlib.pyplot as plt
import numpy as np

from benchmark_functions import discrete_simulated_bifurcation_gpu, balistic_simulated_bifurcation
from IPython.utils.io import capture_output
from tqdm import tqdm
from benchmark_data import read_instance, chimera_data, pegasus_data, ROOT, zephyr_data
from math import inf



for architecture in ["chimera", "pegasus", "zephyr"]:
    data = chimera_data if architecture == "chimera" else pegasus_data if architecture == "pegasus" else zephyr_data

    gaps = []
    times = []
    for instance in tqdm(data, desc=f"Zbieranie danych dla topologii {architecture}"):
        J, h = read_instance(instance.path, convention="minus_half")
        J2 = cp.asarray(J, dtype=cp.float32)
        h2 = cp.asarray(h, dtype=cp.float32)
        # kompilacja 
        with capture_output() as captured:
            states, energies = discrete_simulated_bifurcation_gpu(J2, h2, 100, 0.25, 2**5)

        result = inf
        gap = inf
        elapsed = 0
        count = 0
        best_result = 0
        start = time.time()
        with capture_output() as captured:
            states, energies = discrete_simulated_bifurcation_gpu(J2, h2, 5000, 0.5, 2**11, c_0_scaling=0.7)
        result = min(energies).item()
        checkpoint = time.time()
        elapsed = checkpoint - start
        gap = (instance.best_energy - result)/(instance.best_energy)

        gaps.append(gap)    
        times.append(elapsed)



    with open(os.path.join(ROOT, "wyniki", f"sbm_our_{architecture}_results.pkl"), "wb") as f:
        data = (times, gaps)
        pickle.dump(data, f)



Zbieranie danych dla topologii chimera:   0%|          | 0/5 [00:00<?, ?it/s]


CUDARuntimeError: cudaErrorInsufficientDriver: CUDA driver version is insufficient for CUDA runtime version

In [4]:
# Tworzenie danych dla naszej implementacji wyczerpującego przeszukiwania na GPU

import time
import os
import pickle
import cupy as cp

from benchmark_functions import brute_force_gpu
from tqdm import tqdm
from IPython.utils.io import capture_output
from benchmark_data import ROOT

# Kompilacja
Q = cp.random.uniform(-1, 1, size=(10, 10), dtype=cp.float32)
Q = cp.triu(Q)
with capture_output() as captured:
    brute_force_gpu(Q, 10)

times = []
for n in tqdm(range(10, 41)):
    Q = cp.random.uniform(-1, 1, size=(n, n), dtype=cp.float32)
    Q = cp.triu(Q)
    start = time.time()
    with capture_output() as captured:
        brute_force_gpu(Q, num_states=10, sweep_size_exponent=min(n, 21), threadsperblock=512)
    end = time.time()
    times.append(end - start)

with open(os.path.join(ROOT, "wyniki", "bruteforce_results.pkl"), "wb") as f:
    pickle.dump(times, f)

CUDARuntimeError: cudaErrorInsufficientDriver: CUDA driver version is insufficient for CUDA runtime version

In [5]:
# Tworzenie danych dla naiwnej implementacji na CPU

import time
import os
import pickle
import numpy as np

from benchmark_functions import brute_force_naive
from tqdm import tqdm
from IPython.utils.io import capture_output
from benchmark_data import ROOT


times = []
n_range = list(range(10, 27))
for n in tqdm(n_range, desc="Wyczerpujące przeszukiwanie dla różnych n"):

    J = np.triu(np.random.uniform(-1, 1, size=(n, n)), k=1)  # losowa gęsta macierz górnotrójkątna
    h = np.random.uniform(-1, 1, size=n)  # losowy wektor
    start = time.time()
    with capture_output() as captured:
        state, energy = brute_force_naive(J, h)
    end = time.time()
    elapsed = end - start
    times.append(elapsed)

with open(os.path.join(ROOT, "wyniki", "bruteforce_cpu_results.pkl"), "wb") as f:
    pickle.dump(times, f)

Wyczerpujące przeszukiwanie dla różnych n: 100%|██████████| 17/17 [20:19<00:00, 71.75s/it] 
